In [ ]:
%pip install statsmodels

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.metrics import silhouette_score, mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create output directories
import os
os.makedirs('plots', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

print("=" * 80)
print("=" * 80)

TASK 1: MACHINE DATA ANALYSIS - CYCLONE SENSOR DATA
